In [3]:
from PIL import Image
from ultralytics import YOLO
import json
import numpy as np
import matplotlib.path as mpltPath
import csv
import matplotlib.pyplot as plt
import sys
import matplotlib.colors as mcolors
import cv2
import statistics
import math
import os

In [5]:
model_path = r"S:\Phys\FIV942 Adipo\Datasets\02h9\YO 512 0320 Snow\map75=0607616 YOLOv8n-segml idx=5 ep=52 btch=32 rnd=4670342\weights\best.pt"
val_data = r"S:\Phys\FIV942 Adipo\Adipo2Viz 512"

files = os.listdir(val_data)

model = YOLO(model_path)
predictions = model.predict(val_data, show=False, max_det=1000)

def polygon_area(coords):
    """
    Calculate the area of a polygon given its vertices using numpy.
    :param coords: A numpy array of shape (n, 2), where n is the number of vertices.
    :return: The area of the polygon.
    """
    x = coords[:, 0]
    y = coords[:, 1]
    i = np.arange(len(x))
    # 'shoelace' formula
    # return 0.5*np.abs(np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1)))
    return np.abs(np.sum(x[i-1]*y[i]-x[i]*y[i-1])*0.5)

def find_parents(classes, image):
    class0_indices = np.where(classes == 0)[0]
    class1_indices = np.where(classes == 1)[0]
    
    class1_centers = image.boxes.xywh[class1_indices]
    class0_centers = image.boxes.xywh[class0_indices]
    
    class1_parents = []
    for class1_coord in class1_centers.numpy():
                # print("the class1 instance: ", class1_coord)
                
                droplet_x = class1_coord[0]
                droplet_y = class1_coord[1]
                # print("droplet x, y coords: ", droplet_x, droplet_y)
                min_distance_from_cells = math.inf
                parent_cell_index = 0
                for i, class0_coord in enumerate(class0_centers.numpy()):
                    # print("the class0 instance: ", class0_coord)
                    
                    cell_x = class0_coord[0]
                    cell_y = class0_coord[1]
                    # print("cell x, y coords: ", cell_x, cell_y)
                    point1 = np.array((droplet_x, droplet_y))
                    point2 = np.array((cell_x, cell_y))
                    dist = np.linalg.norm(point1 - point2)
                    # print("dist from class0 instance: ", dist)
                    if dist < min_distance_from_cells:
                        min_distance_from_cells = dist
                        parent_cell_index = i
                        
                # print("parent cell index: ", parent_cell_index)        
                class1_parents.append(class0_indices[parent_cell_index])
    return class1_parents

def find_mask_intensities(img_mask_coords, image_path):
    image_array = np.array(Image.open(image_path))
    pixel_intensity_data = []
    for mask_outline in range(len(img_mask_coords)):
        vertices = img_mask_coords[mask_outline]

        
        # Assuming image_array is a 2D numpy array for a grayscale image
        # For a color image, you need to decide how to handle the color channels

        # Create a mesh grid of coordinate values
        xx, yy = np.meshgrid(np.arange(image_array.shape[1]), np.arange(image_array.shape[0]))

        # Flatten the grid arrays
        x_flat = xx.flatten()
        y_flat = yy.flatten()

        # Create a list of (x, y) points from the flattened grid
        class1_points = np.vstack((x_flat, y_flat)).T

        # Create a path object from the vertices
        polygon_path = mpltPath.Path(vertices)

        # Use the path object to create a mask
        inside_polygon = polygon_path.contains_points(class1_points)

        # Reshape the mask back to the image shape
        mask = inside_polygon.reshape(xx.shape)

        # Apply the mask to select pixels within the polygon
        # selected_pixels = image_array[mask]

        # Sum the intensities of the selected pixels
        # sum_of_intensities = np.sum(selected_pixels)

        # print("Sum of pixel intensities in the polygonal region:", sum_of_intensities)
        # Initialize an array to hold the sum of intensities for each channel
        sum_of_intensities_per_channel = np.zeros(image_array.shape[2])

        # Iterate over each channel
        for i in range(image_array.shape[2]):
            # Apply the mask to the current channel and sum the intensities
            selected_pixels = image_array[:, :, i][mask]
            sum_of_intensities_per_channel[i] = np.sum(selected_pixels)

        # print("Sum of pixel intensities in the polygonal region for each channel:", sum_of_intensities_per_channel)
        pixel_intensity_data.append(sum_of_intensities_per_channel)
        
    return pixel_intensity_data

def find_box_intensities(img_data, image_path):
    img_box_centers = img_data.boxes.xywh.numpy()
    image_array = np.array(Image.open(image_path))
    box_intensity_data = []
    for box_instance in range(len(img_box_centers)):
        bbox_xywh = img_box_centers[box_instance]
        bbox_corners = [[bbox_xywh[0] - bbox_xywh[2], bbox_xywh[1] + bbox_xywh[3]],[bbox_xywh[0] + bbox_xywh[2], bbox_xywh[1] + bbox_xywh[3]]
                    , [bbox_xywh[0] + bbox_xywh[2], bbox_xywh[1] - bbox_xywh[3]], [bbox_xywh[0] - bbox_xywh[2], bbox_xywh[1] - bbox_xywh[3]]]
        vertices = bbox_corners

        
        # Assuming image_array is a 2D numpy array for a grayscale image
        # For a color image, you need to decide how to handle the color channels

        # Create a mesh grid of coordinate values
        xx, yy = np.meshgrid(np.arange(image_array.shape[1]), np.arange(image_array.shape[0]))

        # Flatten the grid arrays
        x_flat = xx.flatten()
        y_flat = yy.flatten()

        # Create a list of (x, y) points from the flattened grid
        class1_points = np.vstack((x_flat, y_flat)).T

        # Create a path object from the vertices
        polygon_path = mpltPath.Path(vertices)

        # Use the path object to create a mask
        inside_polygon = polygon_path.contains_points(class1_points)

        # Reshape the mask back to the image shape
        mask = inside_polygon.reshape(xx.shape)

        # Apply the mask to select pixels within the polygon
        # selected_pixels = image_array[mask]

        # Sum the intensities of the selected pixels
        # sum_of_intensities = np.sum(selected_pixels)

        # print("Sum of pixel intensities in the polygonal region:", sum_of_intensities)
        # Initialize an array to hold the sum of intensities for each channel
        sum_of_intensities_per_channel = np.zeros(image_array.shape[2])

        # Iterate over each channel
        for i in range(image_array.shape[2]):
            # Apply the mask to the current channel and sum the intensities
            selected_pixels = image_array[:, :, i][mask]
            sum_of_intensities_per_channel[i] = np.sum(selected_pixels)

        # print("Sum of pixel intensities in the polygonal region for each channel:", sum_of_intensities_per_channel)
        box_intensity_data.append(sum_of_intensities_per_channel)
        
    return box_intensity_data 

header = ["FilePath", "PolyID","cls","Vertices","Number of Vertices","PolyArea","Sum Inten WV0","SumInten WV1", "SumInten WV2","BBox x","BBox y","BBox w","BBox h",
          "BBox intensity WV0", "BBox intensity WV1","BBox intensity WV2","Conf","PolySum", "PolyAvg", "PolyStDDev", "BoxSum", "BoxAvg", "BoxStDDev", "Parents"]
data = []

for i, image_data in enumerate(predictions):
    print("image ", i)
    print("image name: ", os.listdir(val_data)[i])
    img_box_coords = image_data.boxes.xyxy
    image_path = val_data + "\\" + os.listdir(val_data)[i]
    image_info_toexport = []
    
    contains_masks = True
    if image_data.masks == None:
        contains_masks = False
    print("contains masks? ", contains_masks)
    
    classes = image_data.names
    print("classes: ", classes)
    
    polygon_areas = []
    pixel_intensity_data = []
    if contains_masks:
        polygon_areas = [polygon_area(mask) for mask in image_data.masks.xy]
        print("polygon areas: ", polygon_areas)
        pixel_intensity_data = find_mask_intensities(image_data.masks.xy, image_path)
        print("pixel intensities: ", pixel_intensity_data)
    
    box_intensity_data = find_box_intensities(image_data, image_path)
    print("box intensities: ", box_intensity_data)
    
    print("same number of boxes as polygons? ", len(box_intensity_data) == len(pixel_intensity_data))
        
    class1_parents = []
    if len(classes) == 2:
        class1_parents = find_parents(image_data.boxes.cls.numpy(), image_data)
        print("class1 parents: ", class1_parents)
        
    class1_counter = 0    
    for j in range(len(polygon_areas)):
        cls = predictions[i].boxes[j].cls.numpy().item(0)
        vertices = image_data.masks.xy[j]
        numVertices = len(vertices)
        polyArea = polygon_areas[j]
        bbox_data = predictions[i].boxes[j].xywh.numpy()
        conf = predictions[i].boxes.conf.numpy().item(j)
        polySum = sum([pixel_intensity_data[j][0],pixel_intensity_data[j][1],pixel_intensity_data[j][2]])
        polyAvg = polySum / 3
        polyStdDev = statistics.pstdev([pixel_intensity_data[j][0],pixel_intensity_data[j][1],pixel_intensity_data[j][2]])
        boxSum = sum([box_intensity_data[j][0],box_intensity_data[j][1],box_intensity_data[j][2]])
        boxAvg = boxSum / 3
        boxStdDev = statistics.pstdev([box_intensity_data[j][0],box_intensity_data[j][1],box_intensity_data[j][2]])
        polygon_data = [image_path,j,cls,vertices,numVertices,polyArea,pixel_intensity_data[j][0],pixel_intensity_data[j][1],pixel_intensity_data[j][2]
                        , bbox_data.item(0), bbox_data.item(1), bbox_data.item(2), bbox_data.item(3),box_intensity_data[j][0],box_intensity_data[j][1],box_intensity_data[j][2]
                        ,conf, polySum, polyAvg, polyStdDev, boxSum, boxAvg, boxStdDev]
        if np.where(image_data.boxes.cls.numpy() == 1)[0].__contains__(j):
            # print(j)
            polygon_data.append(class1_parents[class1_counter])
            print("class1 index before increment: ", class1_counter)
            class1_counter += 1
            print("class1 index after increment: ", class1_counter)
        data.append(polygon_data)
        
filename = 'polygon_image_data_test.csv'
with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file)
    csvwriter.writerow(header)
    csvwriter.writerows(data)


image 1/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\001597.bmp: 512x512 64 Cells, 635 Droplets, 66.0ms
image 2/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\001598.bmp: 512x512 25 Cells, 227 Droplets, 54.5ms
image 3/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\001599.bmp: 512x512 16 Cells, 125 Droplets, 52.5ms
image 4/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\FIV942P3-5_FIV942P3.A - 4_17_0_0.bmp: 512x512 14 Cells, 88 Droplets, 53.0ms
image 5/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\FIV942P3-5_FIV942P3.D - 2_25_0_0.bmp: 512x512 42 Cells, 352 Droplets, 58.0ms
image 6/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\FIV942P3-5_FIV942P3.D - 4_7_0_0.bmp: 512x512 5 Cells, 26 Droplets, 56.5ms
image 7/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\kath0.bmp: 512x512 91 Cells, 835 Droplets, 53.1ms
image 8/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\kath10.bmp: 512x512 26 Cells, 227 Droplets, 55.0ms
image 9/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\kath20.bmp: 512x512 77 Cells, 618 Droplets, 56.0ms
image 10/10 S:\Phys\FIV942 Adipo\Adipo2Viz 512\kath21

In [6]:
data

[['S:\\Phys\\FIV942 Adipo\\Adipo2Viz 512\\001597.bmp',
  0,
  1.0,
  array([[        214,          21],
         [        213,          22],
         [        213,          24],
         [        212,          25],
         [        212,          27],
         [        213,          28],
         [        213,          30],
         [        214,          31],
         [        215,          31],
         [        216,          32],
         [        219,          32],
         [        220,          31],
         [        222,          31],
         [        223,          30],
         [        224,          30],
         [        224,          29],
         [        225,          28],
         [        226,          28],
         [        226,          27],
         [        227,          26],
         [        227,          25],
         [        226,          24],
         [        226,          23],
         [        225,          22],
         [        224,          22],
        